In [ ]:
# Cargar variables de entorno desde .env
from dotenv import load_dotenv
import os
load_dotenv()  # Esto cargará las variables definidas en el archivo .env

# **Construir un agente**

Langchain posibilita la creación de [agentes](https://python.langchain.com/docs/concepts/agents/), o sistemas que usan [LLMS](https://python.langchain.com/docs/concepts/chat_models/) como motores de razonamiento para determinar qué acciones tomar y las entradas necesarias para realizar la acción. Después de ejecutar acciones, los resultados se pueden volver a alimentar a la LLM para determinar si se necesitan más acciones o si está bien terminar. Esto a menudo se logra a través del llamado de herramientas [tool-calling](https://python.langchain.com/docs/concepts/tool_calling/).

En este tutorial construiremos un agente que pueda interactuar con un motor de búsqueda. Podrá hacer preguntas de este agente, verlo llamar a la herramienta de búsqueda y tener conversaciones con ella.

In [13]:
!pip install langchain-core langgraph>0.2.27 langgraph-checkpoint-sqlite



# **LangSmith**

Muchas de las aplicaciones que construye con Langchain contendrán múltiples pasos con múltiples invocaciones de llamadas LLM. A medida que estas aplicaciones se vuelven cada vez más complejas, se vuelve crucial poder inspeccionar lo que está sucediendo exactamente dentro de su cadena o agente. La mejor manera de hacer esto es con [LangSmith](https://smith.langchain.com/).

Después de registrarse en el enlace de arriba, asegúrese de establecer sus variables de entorno para comenzar a registrar las trazas:




```
export LANGSMITH_TRACING="true"
export LANGSMITH_API_KEY="..."
export LANGSMITH_PROJECT="default" # or any other project name
```



O, si en un cuaderno, puede configurarlos con:




In [1]:
import getpass
import os
try:
    # load environment variables from .env file (requires `python-dotenv`)
    from dotenv import load_dotenv
    load_dotenv()
except ImportError:
    pass
os.environ["LANGSMITH_TRACING"] = "true"
if "LANGSMITH_API_KEY" not in os.environ:
    os.environ["LANGSMITH_API_KEY"] = getpass.getpass(
        prompt="Enter your LangSmith API key (optional): "
    )
if "LANGSMITH_PROJECT" not in os.environ:
    os.environ["LANGSMITH_PROJECT"] = getpass.getpass(
        prompt='Enter your LangSmith Project Name (default = "pr-glossy-thought-54"): '
    )
    if not os.environ.get("LANGSMITH_PROJECT"):
        os.environ["LANGSMITH_PROJECT"] = "pr-glossy-thought-54"

Enter your LangSmith API key (optional): ··········
Enter your LangSmith Project Name (default = "pr-glossy-thought-54"): ··········


# **Tavily**

Para instalar Tavily:

In [2]:
%pip install -U langchain-tavily

[Tavily](https://python.langchain.com/docs/integrations/tools/tavily_search/) es un motor de búsqueda. Para usarlo, deberá obtener y establecer una clave API:




bash
export TAVILY_API_KEY="..."





O, si está en un cuaderno, puede configurarlo con:




In [3]:
import getpass
import os
os.environ["TAVILY_API_KEY"] = getpass.getpass()

··········




# **Definir herramientas**

Primero necesitamos crear las herramientas que queremos usar. Nuestra principal herramienta de elección será [Tavily](https://python.langchain.com/docs/integrations/tools/tavily_search/) - Un motor de búsqueda. Podemos usar el [Langchain-Tavily](https://pypi.org/project/langchain-tavily/) [paquete de integración](https://python.langchain.com/docs/concepts/architecture/#integration-packages) dedicado para usar fácilmente el motor de búsqueda Tavily como herramienta con Langchain.




In [5]:
from langchain_tavily import TavilySearch
search = TavilySearch(max_results=2)
search_results = search.invoke("Qué tiempo hace hoy en Bilbao?")
print(search_results)
# Se pueden crear otras herramientas si se quiere.
# Una vez que sabemos que esta funciona, la ponemos en una lista y la añadiremos al modelo más tarde.
tools = [search]

{'query': 'Qué tiempo hace hoy en Bilbao?', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'url': 'https://www.eltiempo.es/bilbao.html?v=por_hora', 'title': 'El Tiempo en Bilbao, Vizcaya a 14 días - Previsión por horas', 'content': 'El tiempo en Bilbao · Hoy. 11:00. Icono. 25°. 0 mm. 0%. 6 km/h. 12:00. Icono. 27°. 0 mm. 0%. 7 km/h. 13:00. Icono. 28°. 0 mm · Mañana. 00:00. Icono. 20°. 0 mm. 20', 'score': 0.853046, 'raw_content': None}, {'url': 'https://www.clima.com/espana/euskadi/bilbao', 'title': 'Clima en Bilbao hoy y pronóstico del tiempo a 14 días', 'content': 'Viento. 16 km/h. Ráfagas. 23 km/h. Lluvia. 0 mm. Nieve. 0 cm. Nubes. 55%. Prob. de precipitación. 20%. Radiación UV. -. Humedad. 76%. Presión. 1020 hPa.', 'score': 0.8005209, 'raw_content': None}], 'response_time': 1.67}




En muchas aplicaciones, es posible que desee definir herramientas personalizadas. Langchain admite la creación de herramientas personalizadas a través de funciones de Python y otros medios. Consulte a guíal [Cómo crear herramientas](https://python.langchain.com/docs/how_to/custom_tools/) para obtener más detalles.



# **Uso de modelos del lenguaje**

Langchain admite muchos modelos de idioma diferentes que puede usar indistintamente: ¡seleccione el que desea usar a continuación!

In [ ]:
%pip install -qU langchain-groq

In [7]:
import getpass
import os
if not os.environ.get("GROQ_API_KEY"):
  os.environ["GROQ_API_KEY"] = getpass.getpass("Enter API key for Groq: ")
from langchain.chat_models import init_chat_model
model = init_chat_model("llama3-8b-8192", model_provider="groq")

Enter API key for Groq: ··········




Puede llamar al modelo de idioma pasando una lista de mensajes. Por defecto, la respuesta es un objeto string `content`.




In [8]:
query = "Hola!"
response = model.invoke([{"role": "user", "content": query}])
response.text()

'Hola! ¡Bienvenido! ¿Cómo estás?'



Ahora podemos ver cómo permitir que este modelo realice llamadas de herramientas. Para habilitarlas podemos usar `.bind_tools` para dar al modelo de idioma conocimiento de estas herramientas




In [9]:
model_with_tools = model.bind_tools(tools)



Ahora podemos llamar al modelo. Primero llamemos con un mensaje normal y veamos cómo responde. Podemos mirar ambos campos `content` y`tool_calls`.




In [10]:
query = "Me llamo Aitor y vivo en Bilbao"
response = model_with_tools.invoke([{"role": "user", "content": query}])
print(f"Message content: {response.text()}\n")
print(f"Tool calls: {response.tool_calls}")

Message content: ¡Hola Aitor! Me alegra conocerte. Bilbao es una ciudad hermosa, ¿qué te gusta hacer allí? ¿Tienes algún lugar favorito o actividad que te guste hacer en tu ciudad natal?

Tool calls: []




Ahora, intentemos llamarlo con alguna entrada que esperaría que se llame a una herramienta.




In [11]:
query = "Busca cuál va a ser el tiempo de este fin de semana en Bilbao"
response = model_with_tools.invoke([{"role": "user", "content": query}])
print(f"Message content: {response.text()}\n")
print(f"Tool calls: {response.tool_calls}")

Message content: 

Tool calls: [{'name': 'tavily_search', 'args': {'query': 'weather forecast Bilbao this weekend', 'search_depth': 'advanced', 'time_range': 'week', 'topic': 'general'}, 'id': 'rqtda7b39', 'type': 'tool_call'}]




Podemos ver que ahora no hay contenido de texto, ¡pero hay una llamada de herramienta! Quiere que llamemos a la herramienta de búsqueda de Tavily.

Esto aún no está llamando a esa herramienta, solo nos está diciendo que lo puede hacer. Para llamarlo realmente, queremos crear nuestro agente.

# **Crea el agente**

Ahora que hemos definido las herramientas y el LLM, podemos crear el agente. Estaremos usando [Langgraph](https://python.langchain.com/docs/concepts/architecture/#langgraph) para construir el agente. Actualmente, estamos utilizando una interfaz de alto nivel para construir el agente, pero lo bueno de Langgraph es que esta interfaz de alto nivel está respaldada por una API de bajo nivel y altamente controlable en caso de que desee modificar la lógica del agente.

Ahora, podemos inicializar el agente con el LLM y las herramientas.

Tenga en cuenta que le estamos pasando `model`, no `model_with_tools`. Eso es porque `create_react_agent`llamará `.bind_tools`para nosotros debajo del capó.




In [14]:
from langgraph.prebuilt import create_react_agent
agent_executor = create_react_agent(model, tools)



**Referencia de API:** [create_react_agent](https://langchain-ai.github.io/langgraph/reference/prebuilt/#langgraph.prebuilt.chat_agent_executor.create_react_agent)

# **Ejecutar el agente**

¡Ahora podemos ejecutar el agente con algunas consultas! Tenga en cuenta que por ahora, todos estos son **apátrida** Consultas (no recordará las interacciones anteriores). Tenga en cuenta que el agente devolverá el **final** Estado al final de la interacción (que incluye cualquier entrada, veremos más adelante sobre cómo obtener solo las salidas).

Primero, veamos cómo responde cuando no hay necesidad de llamar a una herramienta:




In [15]:
input_message = {"role": "user", "content": "Hola!"}
response = agent_executor.invoke({"messages": [input_message]})
for message in response["messages"]:
    message.pretty_print()

================================ Human Message =================================

Hola!
================================== Ai Message ==================================

<hello>
{
  "response": "¡Hola! Soy un asistente automatizado. ¿En qué puedo ayudarte hoy?"
}
</hello>




Para ver exactamente lo que está sucediendo debajo del capó (y para asegurarse de que no llame una herramienta) podemos echar un vistazo al [Trace de Langsmith](https://smith.langchain.com/public/28311faa-e135-4d6a-ab6b-caecf6482aaa/r)

Probémoslo ahora en un ejemplo en el que debería invocar la herramienta




In [16]:
input_message = {"role": "user", "content": "Busca el tiempo que va a hacer este sábado en Bilbao, España"}
response = agent_executor.invoke({"messages": [input_message]})
for message in response["messages"]:
    message.pretty_print()

================================ Human Message =================================

Busca el tiempo que va a hacer este sábado en Bilbao, España
================================== Ai Message ==================================
Tool Calls:
  tavily_search (tzg97n0x8)
 Call ID: tzg97n0x8
  Args:
    query: weather Bilbao Saturday
    search_depth: advanced
    time_range: day
    topic: general
================================= Tool Message =================================
Name: tavily_search

{"query": "weather Bilbao Saturday", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://www.timeanddate.com/weather/spain/bilbao/hourly", "title": "Hourly forecast for Bilbao, Vizcaya, Spain - Weather - Time and Date", "content": "| 12:00 pm |  | 73 °F | Overcast. | 74 °F | 4 mph | ↑ | 76% | 3% | 0.00\" (snow) 0.00\" (rain) |\n|  |  |  |  |  |  |  |  |  |  |\n| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |\n| \\ Updated Thursday, July 17, 2025 9:42:2



Podemos ver el [Trace de Langsmith](https://smith.langchain.com/public/f520839d-cd4d-4495-8764-e32b548e235d/r) Para asegurarse de que esté llamando a la herramienta de búsqueda de manera efectiva.

# Streaming

Hemos visto cómo se puede llamar al agente con `.invoke`Para obtener una respuesta final. Si el agente ejecuta múltiples pasos, esto puede llevar un tiempo. Para mostrar un progreso intermedio, podemos transmitir mensajes a medida que ocurren.




In [17]:
for step in agent_executor.stream({"messages": [input_message]}, stream_mode="values"):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Busca el tiempo que va a hacer este sábado en Bilbao, España
================================== Ai Message ==================================
Tool Calls:
  tavily_search (pdfgznd4e)
 Call ID: pdfgznd4e
  Args:
    query: weather Bilbao Saturday
    search_depth: advanced
    time_range: day
    topic: general
================================= Tool Message =================================
Name: tavily_search

{"query": "weather Bilbao Saturday", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://www.timeanddate.com/weather/spain/bilbao/hourly", "title": "Hourly forecast for Bilbao, Vizcaya, Spain - Weather - Time and Date", "content": "| 12:00 pm |  | 73 °F | Overcast. | 74 °F | 4 mph | ↑ | 76% | 3% | 0.00\" (snow) 0.00\" (rain) |\n|  |  |  |  |  |  |  |  |  |  |\n| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |\n| \\ Updated Thursday, July 17, 2025 9:42:2



# **Streaming tokens**

Además de devolver el flujo de mensajes, también es útil devolver los tokens. Podemos hacer esto especificando `stream_mode="messages"`.

> ℹ️ A continuación usamos `message.text()`, que requiere `langchain-core>=0.3.37`.
>




In [18]:
for step, metadata in agent_executor.stream(
    {"messages": [input_message]}, stream_mode="messages"
):
    if metadata["langgraph_node"] == "agent" and (text := step.text()):
        print(text, end="|")

Based| on| the| output| of| the| tool| call| "|g|080|13|b|0|x|",| I| can| see| that| the| query| "|weather| Bil|bao| Saturday|"| yielded| some| results|.| However|,| the| output| is| quite| lengthy| and| seems| to| be| a| mix| of| weather| information| for| Bil|bao| and| other| cities| in| Spain|,| as| well| as| some| extra| content|.

|To| provide| a| more| concise| answer|,| I| can| extract| the| relevant| information| for| Bil|bao|.| According| to| the| output|,| the| weather| forecast| for| Bil|bao| on| Saturday| is| not| explicitly| provided|,| but| we| can| see| that| the| average| temperature| in| July| is| around| |15|-|25|°C| (|59|-|77|°F|).

|If| you| would| like| a| more| accurate| and| concise| weather| forecast| for| Bil|bao| on| Saturday|,| I| can| suggest| an| alternative| tool| call| or| provide| a| direct| response| based| on| general| weather| patterns| in| the| region|.|



# **Agregar memoria**

Como se mencionó anteriormente, este agente no tiene un estado almacenado. Esto significa que no recuerda las interacciones anteriores. Para darle memoria, debemos pasar en un checkpointer. Al pasar un checkpointer, también tenemos que pasarle un `thread_id` al invocar el agente (para que sepa de qué hilo/conversación reanudar).




In [19]:
from langgraph.checkpoint.memory import MemorySaver
memory = MemorySaver()



**Referencia de API:** [MemorySaver](https://langchain-ai.github.io/langgraph/reference/checkpoints/?_gl=1*1p10qdy*_ga*MTE2NzUyNjc2Ny4xNzUyNjU0NTQ2*_ga_47WX3HKKY2*czE3NTI2NjI1ODkkbzQkZzEkdDE3NTI2NjI3MzYkajYwJGwwJGgw#langgraph.checkpoint.memory.MemorySaver)




In [25]:
agent_executor = create_react_agent(model, tools, checkpointer=memory)
config = {"configurable": {"thread_id": "abc765"}}

In [26]:
input_message = {"role": "user", "content": "Hola! Soy de Bilbao. Qué hora es?"}
for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Hola! Soy de Bilbao. Qué hora es?
================================== Ai Message ==================================

¡Hola! Como soy una inteligencia artificial, no tengo acceso a información en tiempo real sobre la hora actual. Sin embargo, puedo decirte que si estás en Bilbao, España, la hora actual depende de la hora del día y la zona horaria. ¿Podrías ser más específico sobre qué hora te gustaría saber?


In [27]:
input_message = {"role": "user", "content": "Busca el tiempo que va a hacer este sábado en la ciudad donde vivo"}
for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Busca el tiempo que va a hacer este sábado en la ciudad donde vivo
================================== Ai Message ==================================
Tool Calls:
  tavily_search (hvt38ndhx)
 Call ID: hvt38ndhx
  Args:
    query: weather forecast for Bilbao, Spain this Saturday
    search_depth: advanced
    time_range: day
================================= Tool Message =================================
Name: tavily_search

{"query": "weather forecast for Bilbao, Spain this Saturday", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://www.ventusky.com/37.450;-5.780", "title": "Location - Weather Forecast Maps | Ventusky", "content": "# Ventusky: Weather Forecast Maps\n\nHail\n\n# Location\n\n|  |\n| --- |\n| 33 °C |\n|  |\n| Wind  9 km/h |\n\n|  |  |\n| --- | --- |\n| Humidity | 32 % |\n\nCalculated from nearby stations (12:30 2025/07/17)\n\n## Weather for the next 24 hou



Ejemplo [Trace de Langsmith](https://smith.langchain.com/public/fa73960b-0f7d-4910-b73d-757a12f33b2b/r)

Si desea comenzar una nueva conversación, todo lo que tiene que hacer es cambiar el `thread_id`usado




In [28]:
config = {"configurable": {"thread_id": "xyz123"}}
input_message = {"role": "user", "content": "What's my name?"}
for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What's my name?
================================== Ai Message ==================================
Tool Calls:
  tavily_search (3ccc6d6c4)
 Call ID: 3ccc6d6c4
  Args:
    query: What's my name?
================================= Tool Message =================================
Name: tavily_search

{"query": "What's my name?", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://www.youtube.com/watch?v=U0CGsw6h60k&pp=0gcJCfwAo7VqN5tD", "title": "Rihanna - What's My Name? ft. Drake - YouTube", "content": "Rihanna - What's My Name? ft. Drake\n\nRihannaVEVO\n3664749 likes\n1072186442 views\n12 Nov 2010\nREMASTERED IN HD!\nGet Rihanna’s eighth studio album ANTI now:\nDownload on TIDAL: http://smarturl.it/downloadANTI\nStream on TIDAL: http://smarturl.it/streamANTIdlx\nDownload on iTunes: http://smarturl.it/dlxANTI\nDownload on Google Play: http://smarturl.it/ANTIdlxgp\nDownload on

KeyboardInterrupt: 



# **Conclusión**

En este comienzo rápido cubrimos cómo crear un agente simple. Luego hemos mostrado cómo transmitir una respuesta, ¡no solo con los pasos intermedios, sino también los tokens! También hemos agregado memoria para poder tener una conversación coherente. ¡Los agentes son un tema complejo con mucho que aprender!

Para obtener más información sobre los agentes, consulte la documentación de [Langgraph](https://python.langchain.com/docs/concepts/architecture/#langgraph). Ésta tiene su propio conjunto de conceptos, tutoriales y guías de instrucciones.
